## Lecture 4 (Options Markets) Assignment, MTH 9865

Due start of class, October 7, 2015

#### Student: Weiyi Chen

### Question 1 (4 marks)

The current time is Wednesday at 1pm and you see the overnight implied volatility (for 10am expiration on Thursday) trading at 9%. The FX markets are open for trading every hour between now and tomorrow at 10am.

The Federal Reserve Chairwoman is speaking about the economy from 2-3pm, and that event adds an extra 0.5 trading days worth of variance on top of the usual variance for that time period.

What should the overnight implied volatility be at 3pm, all else being equal?


#### Solution:

##### Trading time
First case:
- 21 normal "trading hours" from Wed 1pm to Thu 10am
- Plus 12 extra trading hours from the FRC event
- Totals 33 trading hours

Second case:
- Totals 19 trading hours

##### Calendar time
- Actual days/365 = 1/365

##### Total variance
Get the total variance from implied vol^2 * calendar time
$$ 0.09^2 \times 1/365 = 2.219 \times 10^{-5} $$

##### Trading time variance
Convert that into a trading time vol using trading time vol^2 * trading time = total variance
$$ \text{TT vol} = \sqrt{2.219 \times 10^{-5} / 33} = 8.200 \times 10^{-4} $$

##### New Total Variance
Use new trading time to get new total variance, 
$$ \text{Variance} = \text{(TT vol)}^2 \times \text{(trading time in hours)} $$
Keep the same TT vol as before – we assume this is smooth in calendar time
$$ \text{Variance} = (8.200 \times 10^{-4})^2 \times 19 = 1.278 \times 10^{-5}$$

##### Market-convention implied vol
Convert total variance to market-convention implied vol
$$ \text{Implied vol} = \sqrt{\text{total variance} / \text{calendar time}} = \sqrt{1.278 \times 10^{-5} / (1/365)} = 6.83 \times 10^{-2}$$

Therefore the overnight implied vol at 3pm is 6.83%.

In [8]:
sqrt(sqrt(0.09**2 * 1./365 / 33)**2 * 19 / (1./365))

0.06829082195753952

### Question 2 (3 marks)

In stochastic volatility models, why is there a smile? Describe the genesis of the smile in terms of vega gamma.

Similarly, describe why stochastic volatility models generate a skew, in terms of vega dspot.

#### Solution


##### Smile

Definition: The butterfly (smile) comes from “vega gamma”, the derivative of vega with respect to volatility, which comes from volatility of volatility.

To describe the genesis, we

- buy an option with strike such that vega gamma is large (now long vega and long vega gamma)
- vega hedge with an appropriate amount of ATM option (ATM option has zero vega gamma, so still long vega gamma)
- Now whichever way vol moves we make money
    - Buy high strike or low strike options at the flat-vol price
    - Bids them up and raises their prices, and therefore their implied volatilities: that’s a smile because the effective is the same for high and low strike options

##### Skew

Definition: The convexity that drives risk reversal (skew) value is vega dspot: d(vega)/d(spot), which comes from spot/vol correlation.

To describe the genesis, we

- Buy a high strike option (positive vega dspot, positive vega)
- Sell enough ATM to vage hedge (ATM options has zero vega dspot, still long vega dspot)
- Assume positive spot/vol correlation
    - if spot moves up, then vol moves up, then vega turns positive, we make money
    - if spot moves down, then vol moves down, then vega turns negative, we still make money

Reverse is true if you buy a low strike option, similar genesis follows. With positive spot/vol correlation, traders bid up high strike options and offer on low strike options, which drives the skew. (Even if with negative spot/vol correlation, it leads to a negative skew.)


### Question 3 (2 marks)

Why do most FX shops use a “sticky delta” volatility market model when defining delta for hedging purposes, even though that might not give the most accurate estimate of how implied volatilities, and hence portfolio prices, change when spot moves?


#### Solution

“Sticky delta” is vol-by-delta stays fixed as spot moves, which is most common in FX markets since vol-by-delta is quoting convention.

Most desks use sticky delta, which mostly because they mark vol by delta and want to think about PNL that way. If volatility (as an asset) is relatively liquid then it doesn’t matter much




### Question 4 (4 marks)

Consider an ATM EURGBP option with 0.5y to expiration. Assume the EURGBP ATM volatility is 3.5%, the EURUSD ATM volatility is 8.5%, and the GBPUSD ATM volatility is 7.5%. What is the implied correlation between EURUSD and GBPUSD spots?

EURUSD spot is 1.25 and GBPUSD spot is 1.56; assume zero interest rates.

Use the Black-Scholes vega formula to calculate the vegas of all three options and determine the notionals of EURUSD and GBPUSD options needed to hedge the vegas of 1 EUR notional of the EURGBP option, assuming correlation stays constant.

#### Solution

##### Implied correlation

The cross volatility $σ_x$ can be expressed in terms of the two USD-pair volatilities plus a correlation, 

$$ \sigma_x = \sqrt{\sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2} $$

therefore

$$ \rho = \frac{\sigma_1^2 + \sigma_2^2 - \sigma_x^2}{2\sigma_1\sigma_2} = 0.912 $$

In [23]:
rho = (0.085**2 + 0.075**2 - 0.035**2) / (2*0.085*0.075)
rho

0.9117647058823529

##### Vega calculation

Black-Scholes vaga formula is 

$$ vega = S N'(d_1) \sqrt{T-t}\ $$

where $N'(x)$ denote the standard normal probability density function,

$$ N'(x) = \frac{1}{\sqrt{2\pi}} e^{-\frac{x^2}{2}} $$

and 

\begin{align}
     d_1 &= \frac{1}{\sigma\sqrt{T - t}}\left[\ln\left(\frac{S}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)(T - t)\right] = \frac{\sigma}{2}\sqrt{T - t}
\end{align}

therefore 

$$ vega =  \sqrt{\frac{T-t}{2\pi}} S e^{-\frac{\sigma^2(T-t)}{8}}$$

In [25]:
import math

def vega(S, T, t, sigma):
    return sqrt((T-t)/2.*math.pi) * S * exp(-sigma**2*(T-t)/8.)

sigma_1 = 0.085
sigma_2 = 0.075
sigma_x = 0.035

vega_1 = vega(1.25, 0.5, 0, sigma_1)
vega_2 = vega(1.56, 0.5, 0, sigma_2)
vega_x = vega(1.25/1.56, 0.5, 0, sigma_x)

print("EURUSD", vega_1)
print("GBPUSD", vega_2)
print("EURGBP", vega_x)

EURUSD 1.107283536184776
GBPUSD 1.3820280490535959
EURGBP 0.7100633624208024


Now we've the vegas of three currencies, we are going to determine the notionals of EURUSD and GBPUSD options needed to hedge the vegas of 1 EUR notional of the EURGBP option. The portfolio is like

$$ \Pi = S_x + N_1S_1 + N_2S_2 $$

To hedge vega, we require

$$ \frac{\partial\Pi}{\partial\sigma_1} = vega_x\frac{\partial\sigma_x}{\partial\sigma_1} + N_1\cdot vega_1 + N_2\cdot vaga_2 \frac{\partial\sigma_2}{\partial\sigma_1} = 0 $$

and 

$$ \frac{\partial\Pi}{\partial\sigma_2} = vega_x\frac{\partial\sigma_x}{\partial\sigma_2} + N_2\cdot vega_2 + N_1\cdot vaga_1 \frac{\partial\sigma_1}{\partial\sigma_2} = 0 $$

Since $\sigma_1$ and $\sigma_2$ are independent, we have 

$$\frac{\partial\sigma_2}{\partial\sigma_1} = \frac{\partial\sigma_1}{\partial\sigma_2} = 0$$

we finally derive

$$ N_1 = -\frac{vega_x}{vega_1}\cdot\frac{\sigma_1-\rho\sigma_2}{\sigma_x}, 
   N_2 = -\frac{vega_x}{vega_2}\cdot\frac{\sigma_2-\rho\sigma_1}{\sigma_x} $$



In [26]:
N_1 = -(vega_x/vega_1) * (sigma_1-rho*sigma_2) / sigma_x
N_2 = -(vega_x/vega_2) * (sigma_2-rho*sigma_1) / sigma_x
print(N_1, N_2)

-0.3044666638286625 0.03669882940235391


The final answer is 

$$ N_1 = -0.304, N_2 = 0.0367 $$

### Question 5 (10 marks)

In this question you will look at implied correlations and see how much moves in implied correlation contribute to moves in cross volatility, versus moves in the underlying USD-pair volatilities.

#### Solution

In [35]:
"""
Copyright:   Copyright (C) 2015 Baruch College FX modeling - All Rights Reserved
Description: Analysis on implied correlations and see how much moves in implied 
             correlation contribute to moves in cross volatility, versus moves in 
             the underlying USD-pair volatilities.
Author:      Weiyi Chen
"""

# python imports
import datetime

# 3rd party imports
import numpy
import pandas

def impliedCorr_analytics(pairx, pair1, pair2, tenor, sign, startDate, endDate):
    '''
    Function that construct two DataFrames: one holding day-to-day changes in the 
    cross ATM volatility, and one holding differences between the predicted cross 
    volatility (assuming the implied correlation from the day before) and the true 
    cross volatility. The function should print out statistics on both those series.

    
    :param pairx:     name of the cross pair (as 'AUDJPY') 
    :type  pairx:     string
    :param pair1:     name of the first pair (as 'AUDUSD') 
    :type  pair1:     string
    :param pair2:     name of the second pair (as 'USDJPY') 
    :type  pair2:     string
    :param tenor:     a string tenor (like '3m')
    :type  tenor:     string
    :param sign:      a flag to define whether the cross spot is the product or the ratio 
                      of the two USD spots (which affects the sign of the correlation)
    :type  sign:      bool, True if ratio, False if product
    :param startDate: the start date of the historical window
    :type  startDate: datetime
    :param endDate:   the end date of the historical window
    :type  endDate:   datetime
    
    '''
    
    # It should start by loading the data for the ATM implied volatility for the three 
    # tenors from the spreadsheet into pandas DataFrames 

    df = pandas.read_excel('fx_vol_data.xlsx')
    df = df[(df['Date']>=startDate) & (df['Date']<=endDate)]
    dfx = df[pairx+' '+tenor]
    df1 = df[pair1+' '+tenor]
    df2 = df[pair2+' '+tenor]

    # and then calculate a pandas DataFrame of implied correlations.

    impliedCorr = ((dfx**2 - df1**2 - df2**2) / (2.*df1*df2)) * (1-sign)
    
    # The next step: use the correlation from date i, along with the implied volatilities 
    # for the USD pairs on date i+1, to predict the cross volatility on date i+1. Do this 
    # with the pandas DataFrames you have already created.

    dfx_predicted = numpy.sqrt(df1**2 + df2**2 + 2*impliedCorr.shift(1)*df1*df2)
    
    # Finally, construct two DataFrames: one holding day-to-day changes in the cross ATM 
    # volatility, and one holding differences between the predicted cross volatility 
    # (assuming the implied correlation from the day before) and the true cross volatility.

    diffs_1d  = dfx - dfx.shift(1)
    diffs_err = dfx_predicted - dfx
    
    # The function should print out statistics on both those series.
    
    print('Statistics for day-to-day changes in the cross ATM vol:')
    print(diffs_1d.describe())
    print()
    
    print('Statistics for difference between the predicted and the true cross vol:')
    print(diffs_err.describe())
    print()

def main():    
    for tenor in ['1w','1m','6m','1y']:
        print('Tenor:',tenor)
        impliedCorr_analytics(
            pairx = 'AUDJPY',
            pair1 = 'AUDUSD',
            pair2 = 'USDJPY',
            tenor = tenor,
            sign = False,
            startDate = datetime.date(2007,1,1),
            endDate = datetime.date(2013,5,31),
        )
        print()

if __name__=='__main__':
    main()

Tenor: 1w
Statistics for day-to-day changes in the cross ATM vol:
count    1621.000000
mean        0.005345
std         1.795480
min       -15.880000
25%        -0.580000
50%         0.000000
75%         0.505000
max        18.805000
Name: AUDJPY 1w, dtype: float64

Statistics for difference between the predicted and the true cross vol:
count    1.621000e+03
mean     4.047207e-03
std      1.186038e+00
min     -1.392378e+01
25%     -3.995371e-01
50%     -1.776357e-15
75%      4.087137e-01
max      1.495141e+01
dtype: float64


Tenor: 1m
Statistics for day-to-day changes in the cross ATM vol:
count    1621.000000
mean        0.004625
std         1.096523
min        -8.627500
25%        -0.387500
50%        -0.020000
75%         0.302500
max        12.547500
Name: AUDJPY 1m, dtype: float64

Statistics for difference between the predicted and the true cross vol:
count    1621.000000
mean       -0.000849
std         0.742401
min        -7.311626
25%        -0.189330
50%        -0.002107
75%

Statistics for day-to-day changes in the cross ATM vol:

|      | 1w         | 1m         | 6m         | 1y         |
|------|------------|------------|------------|------------|
| mean | 0.005345   | 0.004625   | 0.003956   | 0.003694   |
| std  | 1.795480   | 1.096523   | 0.575049   | 0.431407   |
| min  | -15.880000 | -8.627500  | -3.722500  | -2.482500  |
| max  | 18.805000  | 12.547500  | 4.922500   |  3.717500  |

Statistics for difference between the predicted and the true cross vol:

|      | 1w         | 1m         | 6m         | 1y         |
|------|------------|------------|------------|------------|
| mean | 4.047207e-03  | -0.000849 | -0.000562  | -0.000429  |
| std  | 1.186038e+00  | 0.742401  | 0.383226   | 0.315963   |
| min  | -1.392378e+01 | -7.311626 | -3.415851  | -3.265282  |
| max  | 1.495141e+01  | 7.866179  | 4.922500   |  4.605420  |

Comments:

As the tenor goes up, both standard deviations as well as the maximum and minimum deviations of the predicted difference and daily changes goes down. It's mainly because the market becomes more stable when the tenor goes up. And our predicted cross vol would become more accurate.